# <h2 style= "text-align:center"> <b>Đồ án thực hành </b> </h2>
# <h2 style='text-align:center;font-size:32px'><b>Đề tài</b>: Natural Language Processing with Disaster Tweets</h1>
# <h2 style='text-align:center;font-size:32px'><b>Task 3-4</b>: Khám phá dữ liệu - Tiền xử lý dữ liệu</h1>

<hr/>

### Install libraries

In [246]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [247]:
pip install --upgrade nbformat

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [248]:
pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [249]:
pip install unidecode

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### Import libraries

In [250]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import PorterStemmer
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import string
from nltk import word_tokenize
import nltk
from unidecode import unidecode

In [251]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Đọc dữ liệu

In [252]:
data_train=pd.read_csv('data/train.csv')
data_test=pd.read_csv('data/test.csv')

In [253]:
# data_train
data_train.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [254]:
# data_test
data_test.head(5)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


<hr/>

## 3. Khám phá dữ liệu

#### 3.1 Shape

+ Tập train có 7613 dòng và 5 cột
+ Tập test có 3263 hàng và 4 cột

In [255]:
data_train.shape

(7613, 5)

In [256]:
data_test.shape

(3263, 4)

#### 3.2 Ý nghĩa mỗi cột

Dữ liệu gồm 5 cột và mỗi cột mạng một ý nghĩa riêng:
+ id: một mã định danh duy nhất cho mỗi tweet
+ text: văn bản của tweet
+ location: vị trí gửi tweet từ đó (có thể để trống)
+ keyword: một từ khóa cụ thể từ tweet (có thể để trống)
+ target: chỉ trong train.csv , điều này biểu thị liệu một tweet có phải là về một thảm họa thực sự (1) hay không ( 0)

In [257]:
data_train.columns

Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')

#### 3.3 Ý nghĩa các dòng

Mỗi dòng là một tweet dùng để phân loại (dán nhãn)

#### 3.4 Kiểm tra dữ liệu lặp

In [258]:
data_train[data_train.duplicated()].shape[0]

0

In [259]:
data_test[data_test.duplicated()].shape[0]

0

Nhận xét: Cả dữ liệu train và test để không dòng nào bị lặp

#### 3.5 Thông tin tổng quát mỗi cột

In [260]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


Nhận xét: Ở tập train cột keyword có 61 ô bị thiếu và ở cột location có 2533 ô bị thiếu

In [261]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB


Nhận xét: Ở tập test  cột keyword có 26 ô bị thiếu và ở cột location có 1105 ô bị thiếu

#### 3.6. Ảnh hưởng của keywork đối với target

Số lượng keyword

In [262]:
data_train['keyword'].unique().__len__()

222

In [263]:
count = data_train.groupby(by = ["keyword"])['target'].value_counts().reset_index(name='count')
split = count[count['target']==1].set_index('keyword').join(
    count.groupby(by = ["keyword"])["count"].sum().reset_index(name='sum').set_index('keyword')
    ,on="keyword",how='left').fillna(0)
percent = split.join((split['count']/split['sum']).reset_index(name="percent")
                     .set_index("keyword")).reset_index().sort_values(by = ['percent'],ascending=False)
#percent

In [264]:
fig = px.bar(percent[:15], x="percent", y='keyword',color="keyword",orientation='h',title="Top % 15 keywork có target==1 nhiều nhất")
fig.show()
fig = px.bar(percent[-15:-1], x="percent", y='keyword',color="keyword",orientation='h',title="Top % 15 keywork có target==1 ít nhất")
fig.show()

#### 3.7. Ảnh hưởng của location đối với target

Số lượng location

In [265]:
data_train['location'].unique().__len__()

3342

In [266]:
count = data_train.groupby(by=['location']).count().sort_values(by='target',ascending=False).reset_index()
fig = px.bar(count[:15], x="target", y='location',color="location",orientation='h',title="Top 15 location có nhiều tweet nhiều nhất")
fig.show()

In [267]:
count = data_train[data_train['location'].isin(count['location'][:20])].groupby(by = ["location"])['target'].value_counts().reset_index(name='count')
split = count[count['target']==1].set_index('location').join(
    count.groupby(by = ["location"])["count"].sum().reset_index(name='sum').set_index('location')
    ,on="location",how='left').fillna(0)
percent = split.join((split['count']/split['sum']).reset_index(name="percent")
                     .set_index("location")).reset_index().sort_values(by = ['percent'],ascending=False)
#percent

In [268]:
fig = px.bar(percent[:15], x="percent", y='location',color="location",orientation='h',title="Top 15 location có nhiều tweet nhất xếp theo số lượng target==1")
fig.show()

#### 3.8. Khám phá các comment nhiều link, hashtag, @.

In [269]:
from tqdm.notebook import tqdm
tqdm.pandas()

def split_info(data):
    regex_link = "https{0,1}://[^\s]+"
    data['link'] = data['text'].progress_apply(lambda x : re.findall(regex_link,x))
    data['text'] = data['text'].progress_apply(lambda x : re.sub(regex_link,'',x))
    regex_hashtag = "#[^\s#@]+"
    data['hashtag'] = data['text'].progress_apply(lambda x : re.findall(regex_hashtag,x))
    data['text'] = data['text'].progress_apply(lambda x : re.sub(regex_hashtag,'',x))
    regex_tag = "@[^\s#@]+"
    data['tag'] = data['text'].progress_apply(lambda x : re.findall(regex_tag,x))
    data['text'] = data['text'].progress_apply(lambda x : re.sub(regex_tag,'',x))
    return data
data_train = split_info(data_train)
data_test = split_info(data_test)

  0%|          | 0/7613 [00:00<?, ?it/s]

  0%|          | 0/7613 [00:00<?, ?it/s]

  0%|          | 0/7613 [00:00<?, ?it/s]

  0%|          | 0/7613 [00:00<?, ?it/s]

  0%|          | 0/7613 [00:00<?, ?it/s]

...

  0%|          | 0/3263 [00:00<?, ?it/s]

  0%|          | 0/3263 [00:00<?, ?it/s]

  0%|          | 0/3263 [00:00<?, ?it/s]

  0%|          | 0/3263 [00:00<?, ?it/s]

  0%|          | 0/3263 [00:00<?, ?it/s]

Comment chia thành 2x2x2 loại tương ứng 'link','hashtag','tag' có xuất hiện hay không

In [270]:
link_hashtag_tag = data_train[['link','hashtag','tag','target']]
link_hashtag_tag = link_hashtag_tag.progress_apply(lambda x:x.apply(lambda y: not(not(y))))

  0%|          | 0/4 [00:00<?, ?it/s]

In [271]:
link_hashtag_tag = link_hashtag_tag.groupby(by=['link','hashtag','tag'])['target'].value_counts().reset_index(name='count').replace([True,False],['1','0'])
new_index = '('+link_hashtag_tag['link'] +','+link_hashtag_tag['hashtag']+','+link_hashtag_tag['tag']+')'
link_hashtag_tag.index = new_index


fig = px.bar(link_hashtag_tag, x=link_hashtag_tag.index, y='count',color='target',title="(link,hashtag,tag) theo phân loại target")
fig.show()

Ứng với mỗi thuộc tính link,hashtag,tag có cái gì xuất hiện nhiều lần không?

In [272]:
for i in ['link','hashtag','tag'] :
    def get_top(att,target_name,target_val):
        _copy_ = data_train[[att,target_name]]
        _copy_ = [i for x in _copy_[_copy_[target_name]==target_val][att] for i in x]
        _copy_ = {i:_copy_.count(i) for i in list(dict.fromkeys(_copy_))}
        _copy_ = dict(sorted(_copy_.items(), key=lambda item: item[1],reverse=True)[:10])
        _copy_ = pd.DataFrame.from_dict(dict(keys = _copy_.keys(),values = _copy_.values()))
        return _copy_
    _copy_1 = get_top(i,'target',1)
    _copy_0 = get_top(i,'target',0)
    fig = make_subplots(rows=1, cols=2, shared_yaxes=True)
    fig.add_trace(go.Bar(x=_copy_1['keys'], y=_copy_1['values'],name='taget==1'),1, 1)
    fig.add_trace(go.Bar(x=_copy_0['keys'], y=_copy_0['values'],name='taget==0'),1, 2)
    fig.update_layout(coloraxis=dict(colorscale='Bluered_r'), showlegend=True,title={'text': f"{i}, Top phổ biến"})
    fig.show()

#### 3.9. Kiểm tra các nhãn trùng text nhưng khác label

In [273]:
duplicates = data_train.duplicated(subset=['text'],keep = False)
different_labels = data_train[duplicates].groupby('text')['target'].nunique() > 1
mismatched_rows = data_train[(data_train['text'].isin(different_labels.index)) & duplicates]

In [274]:
for text, group in mismatched_rows.groupby('text'):
    majority_label = group['target'].value_counts().idxmax()
    data_train.loc[group.index, 'target'] = majority_label

## 4. Tiền xử lý dữ liệu

+ Như chúng ta thấy ở trên cột keyword, location và text đều là văn bản và đều mang ý nghĩa thể hiện xem dòng tweet đó có mang ý nghĩa tích cực hay không. Vì thế ở đây chúng ta sẽ nối 3 cột lại thành một cột là một đoạn văn bản.
+ Ở tập train điều chúng ta chú ý là 3 cột văn bản và cột label nên chúng ta sẽ tạo ra một dataframe mới gồm 2 cột là text và label
+ Ở tập test thì điều chúng ta quan tâm là cột id và cột văn bản nên dataframe của tập test sẽ là 2 cột id và text 

#### 4.1. Clean "Location" column 

In [275]:
def clean_location(x):
    if x != x:
        return x
    elif x == 'Earth' or x =='Worldwide' or x == 'Everywhere':
        return 'World'
    elif 'New York' in x or 'NYC' in x:
        return 'New York'
    elif 'London' in x:
        return 'London'
    elif 'Mumbai' in x:
        return 'Mumbai'
    elif 'Washington' in x and 'D' in x and 'C' in x:
        return 'Washington DC'
    elif 'San Francisco' in x:
        return 'San Francisco'
    elif 'Los Angeles' in x:
        return 'Los Angeles'
    elif 'Seattle' in x:
        return 'Seattle'
    elif 'Chicago' in x:
        return 'Chicago'
    elif 'Toronto' in x:
        return 'Toronto'
    elif 'Sacramento' in x:
        return 'Sacramento'
    elif 'Atlanta' in x:
        return 'Atlanta'
    elif 'California' in x:
        return 'California'
    elif 'Florida' in x:
        return 'Florida'
    elif 'Texas' in x:
        return 'Texas'
    elif 'United States' in x or 'USA' in x:
        return 'USA'
    elif 'United Kingdom' in x or 'UK' in x or 'Britain' in x:
        return 'UK'
    elif 'Canada' in x:
        return 'Canada'
    elif 'India' in x:
        return 'India'
    elif 'Kenya' in x:
        return 'Kenya'
    elif 'Nigeria' in x:
        return 'Nigeria'
    elif 'Australia' in x:
        return 'Australia'
    elif 'Indonesia' in x:
        return 'Indonesia'
    elif x in percent['location']:
        return x
    else: return 'Others'

In [276]:
data_train['location'] = data_train['location'].apply(lambda x: clean_location(x))
data_test['location'] = data_test['location'].apply(lambda x: clean_location(x))

#### 4.2. Fill NA

In [277]:
for i in ['location','keyword']:
    data_train[i] = data_train[i].fillna(f'NA_{i}')
    data_test[i] = data_test[i].fillna(f'NA_{i}')

#### 4.3. Preprocessing text

In [278]:
def preprocess_text(sen):
    """
    Xử lý đoạn văn bản đầu vào.

    Parameters:
    - sen: Đoạn văn bản cần xử lý.

    Returns:
    - Đoạn văn bản mới sau khi loại bỏ các kí tự không cần thiết.
    """
    sentence =sen.lower()

    stops = stopwords.words('english')
    #print(stops)
    for punc in string.punctuation:
          if punc == '-' : continue
          sentence = sentence.replace(punc, ' ')

    words = word_tokenize(sentence)

    porter = PorterStemmer()
    words = [porter.stem(word) for word in words if word.lower() not in stops]
    sentence = ' '.join(words)
    sentence = re.sub(r'\s+', ' ', sentence)
    sentence = sentence.strip()
    return sentence

#### 4.4. Các kí tự đặc biệt cần được biểu thị rõ ràng

In [279]:
data_train['text'] = data_train['text'].progress_apply(unidecode)
data_test['text'] = data_test['text'].progress_apply(unidecode)

  0%|          | 0/7613 [00:00<?, ?it/s]

  0%|          | 0/3263 [00:00<?, ?it/s]

#### 4.5. Gộp tất cả các cột để train

In [280]:
data_train['extras'] = data_train['link'].apply(' '.join) +' '+ data_train['hashtag'].apply(' '.join) +' '+ data_train['tag'].apply(' '.join)
data_test['extras'] = data_test['link'].apply(' '.join) +' '+ data_test['hashtag'].apply(' '.join) +' '+ data_test['tag'].apply(' '.join)

In [281]:
new_train = data_train[['id','text','extras','target']]
new_test = data_test[['id','text','extras']]

In [282]:
# data_train['new_text']= data_train.apply(lambda row: row['text'] + ' ' + row['extras'],axis = 1)
# data_test['new_text'] = data_test.apply(lambda row: row['text'] + ' ' + row['extras'],axis = 1)

In [283]:
# new_train = pd.DataFrame({'text': data_train['new_text'],'target': data_train['target']})
# new_test = pd.DataFrame({'id': data_test['id'],'text': data_test['text']})

In [284]:
new_train.head()

,id,text,extras,target
0,1,Our Deeds are the Reason of this May ALLAH Fo...,#earthquake,1
1,4,Forest fire near La Ronge Sask. Canada,,1
2,5,All residents asked to 'shelter in place' are ...,,1
3,6,"13,000 people receive evacuation orders in Ca...",#wildfires,1
4,7,Just got sent this photo from Ruby as smoke f...,#Alaska #wildfires,1


In [285]:
new_test.head()

,id,text,extras
0,0,Just happened a terrible car crash,
1,2,"Heard about is different cities, stay safe ev...",#earthquake
2,3,"there is a forest fire at spot pond, geese are...",
3,9,Apocalypse lighting.,#Spokane #wildfires
4,11,Typhoon Soudelor kills 28 in China and Taiwan,


#### 4.6. Write to new file CSV

In [286]:
# train data
new_train.to_csv('data/new_train.csv',index=False)

In [287]:
# test data
new_test.to_csv('data/new_test.csv',index=False)